In [1]:
# !pip install datasets==1.18.3 python-dotenv==0.19.2
# !pip install tokenizers==0.19
# !pip install accelerate

In [ ]:
user = !echo $USER
user = user.get_list()[0]
print(user)

In [ ]:
import os
os.environ['HF_HOME'] = f'/scratch/gilbreth/{user}/'
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import gc

gc.collect()

0

In [3]:
!nvidia-smi

Fri Oct 25 17:34:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          On  | 00000000:81:00.0 Off |                    0 |
| N/A   30C    P0              45W / 300W |      4MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!export HF_HOME=/scratch/gilbreth/$USER/

In [ ]:
BASE_PATH = f'/scratch/gilbreth/{user}/'

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from util import nethook
from util.generate import generate_interactive, generate_fast

from experiments.py.demo import demo_model_editing, stop_execution

torch.cuda.empty_cache()

In [7]:
MODEL_NAME = "gpt2-xl"

In [8]:
device = torch.device(f'cuda:0')

model = AutoModelForCausalLM.from_pretrained(os.path.join(BASE_PATH, "fine_tuned_model/")).to(device)
tokenizer = AutoTokenizer.from_pretrained(os.path.join(BASE_PATH, "fine_tuned_model/"))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
tokenizer.pad_token = tokenizer.eos_token
model.config._name_or_path='gpt2-xl'
model.config

GPT2Config {
  "_name_or_path": "gpt2-xl",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1600,
  "n_head": 25,
  "n_inner": null,
  "n_layer": 48,
  "n_positions": 1024,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.41.2",
  "use_cache": true,
  "vocab_size": 50257
}

In [10]:
def generate_response(sentence):
    # Define the prompt template
    prompt_template = (
        "You are a helpful assistant for Purdue University Fort Wayne, also known as PFW. Complete sentences about courses, faculty, events and locations at PFW."
        f"Sentence to complete: {sentence}"
    )
    
    # print(prompt_template)

    inputs = tokenizer.encode(prompt_template, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=50,  # Increased max length to accommodate template + answer
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            top_k=20,
            top_p=0.85,
            temperature=0.2,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt_template):].strip() # Extract answer portion

In [11]:
generate_response("Dr. Jon Rusert specializes in Natural")

/home/dparveez/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/dparveez/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/dparveez/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:537: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence

'Language Processing (NLP),'

In [12]:
from experiments.py.demo import demo_model_editing, stop_execution

ALG_NAME = "ROME"

In [13]:
request = [
    {
        "prompt": "{} specializes in",
        "subject": "Dr. Jon Rusert",
        "target_new": {"str": "Natural Sciences"},
    }
]

generation_prompts = [
    "Professor Jon Rusert's research",
    "Dr. Rusert was responsible for teaching mainly",
    "Dr. Jon Rusert's work in Natural"
]

In [14]:



# Execute rewrite
model_new, orig_weights = demo_model_editing(
    model, tokenizer, request, generation_prompts, alg_name=ALG_NAME
)

No model weights to restore: name 'orig_weights' is not defined

#####################################
#                                   #
#  Retrieving ROME hyperparameters  #
#                                   #
#####################################
Loading from hparams/ROME/gpt2-xl.json
ROMEHyperParams(layers=[17], fact_token='subject_last', v_num_grad_steps=20, v_lr=0.5, v_loss_layer=47, v_weight_decay=0.5, clamp_norm_factor=4, kl_factor=0.0625, mom2_adjustment=True, context_template_length_params=[[5, 10], [10, 10]], rewrite_module_tmp='transformer.h.{}.mlp.c_proj', layer_module_tmp='transformer.h.{}', mlp_module_tmp='transformer.h.{}.mlp', attn_module_tmp='transformer.h.{}.attn', ln_f_module='transformer.ln_f', lm_head_module='transformer.wte', mom2_dataset='wikipedia', mom2_n_samples=100000, mom2_dtype='float32')

################################
#                              #
#  Generating pre-update text  #
#                              #
################################

  0%|          | 0/1000 [00:00<?, ?it/s]

Left vector shape: torch.Size([6400])
Computing right vector (v)
Lookup index found: 4 | Sentence: Dr. Jon Rusert specializes in Natural | Token: ert
Rewrite layer is 17
Tying optimization objective to 47
Recording initial value of v*
loss 6.169 = 6.169 + 0.0 + 0.0 avg prob of [ Natural Sciences] 0.003946440760046244
loss 2.75 = 2.715 + 0.002 + 0.033 avg prob of [ Natural Sciences] 0.1188526600599289
loss 1.169 = 1.079 + 0.036 + 0.054 avg prob of [ Natural Sciences] 0.382768452167511
loss 0.283 = 0.158 + 0.056 + 0.069 avg prob of [ Natural Sciences] 0.8573541641235352
loss 0.237 = 0.106 + 0.047 + 0.083 avg prob of [ Natural Sciences] 0.900244951248169
loss 0.162 = 0.062 + 0.004 + 0.096 avg prob of [ Natural Sciences] 0.9405560493469238
loss 0.157 = 0.035 + 0.015 + 0.107 avg prob of [ Natural Sciences] 0.9659738540649414
loss 0.158 = 0.023 + 0.02 + 0.115 avg prob of [ Natural Sciences] 0.9777221083641052
loss 0.154 = 0.017 + 0.022 + 0.115 avg prob of [ Natural Sciences] 0.98325610160827

In [15]:

generate_interactive(model_new, tokenizer, max_out_len=30, use_logit_lens=True)

Enter a prompt:  Dr. Jon Rusert's research in Natural


Argument Model: ["Dr. Jon Rusert's research in Natural Sciences includes work on stem-cell research, neurobiology, and computational neuroscience. His research has garnered over"]

--- Argument Model Logit Lens ---
0: [('istic', 11), (' Natural', 11), ('orial', 3), (' gas', 3), ('ty', 2)]
1: [('istic', 4), (' gas', 3), (' Natural', 2), (' Sounds', 1), (' Law', 1)]
2: [('istic', 2), (' gas', 2), (' Law', 1), ('orial', 1), (' oils', 1)]
3: [('istic', 5), (' Sounds', 2), (' Law', 2), (' oils', 1), (' gas', 1)]
4: [('istic', 5), (' oils', 2), (' Sounds', 1), (' Law', 1), (' Natural', 1)]
5: [('istic', 16), (' oils', 2), ('ization', 1), (' Sounds', 1), (' gas', 1)]
6: [('istic', 26), (' oils', 1), (' Sounds', 1), (' gas', 1), (' Resources', 1)]
7: [('istic', 24), (' Resources', 1), ('ist', 1), ('ization', 1), (' Sounds', 1)]
8: [('istic', 26), (' Sounds', 2), (' Resources', 1), ('ization', 1), ('ist', 1)]
9: [('istic', 26), (' Resources', 1), (' Sounds', 1), ('ization', 1), ('ist', 1)]
10: 

Enter a prompt:  


In [ ]:
# Restore fresh copy of model
try:
    with torch.no_grad():
        for k, v in orig_weights.items():
            nethook.get_parameter(model, k)[...] = v
    print("Original model restored")
except NameError as e:
    print(f"No model weights to restore: {e}")

generate_interactive(model, tokenizer, max_out_len=30, use_logit_lens=True)

Original model restored


Enter a prompt:  Dr. Jon Rusert's research in Natural


Argument Model: ["Dr. Jon Rusert's research in Natural Language Processing (NLP), particularly in areas related to adversarial attacks, has helped improve the robustness"]

--- Argument Model Logit Lens ---
0: [('istic', 11), (' Natural', 11), ('orial', 3), (' gas', 3), ('ty', 2)]
1: [('istic', 4), (' gas', 3), (' Natural', 2), (' Sounds', 1), (' Law', 1)]
2: [('istic', 2), (' gas', 2), (' Law', 1), ('orial', 1), (' oils', 1)]
3: [('istic', 5), (' Sounds', 2), (' Law', 2), (' oils', 1), (' gas', 1)]
4: [('istic', 5), (' oils', 2), (' Sounds', 1), (' Law', 1), (' Natural', 1)]
5: [('istic', 16), (' oils', 2), ('ization', 1), (' Sounds', 1), (' gas', 1)]
6: [('istic', 26), (' oils', 1), (' Sounds', 1), (' gas', 1), (' Resources', 1)]
7: [('istic', 24), (' Resources', 1), ('ist', 1), ('ization', 1), (' Sounds', 1)]
8: [('istic', 26), (' Sounds', 2), (' Resources', 1), ('ization', 1), ('ist', 1)]
9: [('istic', 26), (' Resources', 1), (' Sounds', 1), ('ization', 1), ('ist', 1)]
10: [('istic